# Noisy ICD Code Labeling of Clincal Notes

In [ ]:
%load_ext autoreload
%autoreload 2

import json
from pathlib import Path

import numpy as np
from deep_patient_cohorts import NoisyLabeler

## Preprocess the data

In [ ]:
valid = [json.loads(line) for line in Path("../data/MIMIC-III/valid.jsonl").read_text().strip().split("\n")]
texts = [example["text"] for example in valid[:100]]
labels = [example["labels"] for example in valid[:100]]
mlb = MultiLabelBinarizer()
_= mlb.fit_transform(labels)
whitelist = mlb.classes_

## (Noisy) Label the data

In [ ]:
labeler = NoisyLabeler(descriptions="../data/MIMIC-III/D_ICD_DIAGNOSES.csv", whitelist=whitelist)

In [ ]:
noisy_labels = labeler(texts)

In [ ]:
accuracy, abstain_rate = labeler.accuracy(noisy_labels, labels)

In [ ]:
m = list(noisy_labels.values())[0].shape[-1]
for i in range(m):
    print(
        f"LF {i}: Accuracy {np.mean(accuracy[i]) * 100:.2f}%,"
        f" Abstain rate {np.mean(abstain_rate[i]) * 100:.2f}%"
    )